In [ ]:
!pip install yfinance
!pip install PyPortfolioopt

     |████████████████████████████████| 6.3 MB 7.3 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=99dc986e64996385612575a4ac14c0e0c92e7c6c9ac755db4f19a9c4dee735e9
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 60 kB 3.2 MB/s 
     |████████████████████████████████| 2.7 MB 11.2 MB/s 
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [ ]:
assets=["TLT", "TSLA", "FB","^DJI","GC=F"]
df=pd.DataFrame()

In [ ]:
df=yf.download(assets, start="2013-01-01", end="2020-12-31")['Adj Close']

[*********************100%***********************]  5 of 5 completed


In [ ]:
returns=df.pct_change()
returns

,FB,GC=F,TLT,TSLA,^DJI
Date,,,,,
2013-01-02,NaN,NaN,NaN,NaN,NaN
2013-01-03,-0.008214,-0.008413,-0.013550,-0.016686,-0.001580
2013-01-04,0.035650,-0.015295,0.003900,-0.010641,0.003274
2013-01-07,0.022949,-0.001578,0.000422,-0.001744,-0.003790
2013-01-08,-0.012237,0.009723,0.006585,-0.019220,-0.004142
...,...,...,...,...,...
2020-12-23,0.003819,0.004339,-0.006909,0.008808,0.003809
2020-12-24,-0.002648,0.000000,0.003957,0.024444,0.002325
2020-12-28,0.035901,0.001334,0.000445,0.002901,0.006758


In [ ]:
#Kovarians
cov_matrix_annual=returns.cov() * 250
cov_matrix_annual

,FB,GC=F,TLT,TSLA,^DJI
FB,0.117869,0.000575,-0.008153,0.057612,0.028905
GC=F,0.000575,0.024429,0.004890,0.004331,-0.000184
TLT,-0.008153,0.004890,0.018413,-0.010213,-0.009392
TSLA,0.057612,0.004331,-0.010213,0.312612,0.034709
^DJI,0.028905,-0.000184,-0.009392,0.034709,0.030829


In [ ]:
#Pearons korrelation
pear=returns.corr()
pear

,FB,GC=F,TLT,TSLA,^DJI
FB,1.000000,0.010722,-0.175006,0.300130,0.479511
GC=F,0.010722,1.000000,0.230573,0.049563,-0.006687
TLT,-0.175006,0.230573,1.000000,-0.134619,-0.394198
TSLA,0.300130,0.049563,-0.134619,1.000000,0.353558
^DJI,0.479511,-0.006687,-0.394198,0.353558,1.000000


In [ ]:
weights=np.array([0.20,0.20,0.20,0.20,0.20])

In [ ]:
weights

array([0.2, 0.2, 0.2, 0.2, 0.2])

In [ ]:
#Udregning af porteføljens varians
port_variance = np.dot(weights.T, np.dot(cov_matrix_annual, weights))

#Udregning af standafvigelse (volatilitet)
port_volatility=np.sqrt(port_variance)
port_volatility

0.168560357432708

In [ ]:
#Årlige afkast fra porteføljen
portfolioSimpleAnnualReturn = np.sum(returns.mean()*weights) * 253
portfolioSimpleAnnualReturn

0.2585728276712513

In [ ]:
#Forventede årlige afkast
percent_return=str(round(portfolioSimpleAnnualReturn, 2)*100)+"%"

#Volatilitet (risiko)
percent_vol = str(round(port_volatility,2)*100)+"%"

print('Det forventede årlige afkast:' + percent_return)
print('Det forventede volatilitet/risiko:' + percent_vol)


Det forventede årlige afkast:26.0%
Det forventede volatilitet/risiko:17.0%


In [ ]:
#Forventede årlige afkast 
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

#Optimering ud fra maksimal sharpe ratio
ef = EfficientFrontier(mu,S)
weights_ef=ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('FB', 0.22018), ('GC=F', 0.0), ('TLT', 0.51334), ('TSLA', 0.26648), ('^DJI', 0.0)])
Expected annual return: 31.1%
Annual volatility: 18.8%
Sharpe Ratio: 1.55


(0.3106010448763968, 0.18753155970354615, 1.5496114111981205)

In [ ]:
S

,FB,GC=F,TLT,TSLA,^DJI
FB,0.118812,0.000580,-0.008218,0.058073,0.029137
GC=F,0.000580,0.024625,0.004929,0.004366,-0.000185
TLT,-0.008218,0.004929,0.018560,-0.010295,-0.009467
TSLA,0.058073,0.004366,-0.010295,0.315113,0.034987
^DJI,0.029137,-0.000185,-0.009467,0.034987,0.031076
